## Import Packages

In [27]:
import pandas as pd

from sklearn.model_selection import train_test_split

import json

from transformers import BertTokenizer, BertForTokenClassification

from transformers import BertTokenizerFast



import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AdamW, get_linear_schedule_with_warmup

import numpy as np

import random



# 把預訓練好的modelload進來

from tqdm.auto import tqdm



# 设置设备（如果有GPU，则使用GPU）

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



def same_seeds(seed):



    torch.manual_seed(seed)



    if torch.cuda.is_available():



        torch.cuda.manual_seed(seed)



        torch.cuda.manual_seed_all(seed)



    np.random.seed(seed)



    random.seed(seed)



    torch.backends.cudnn.benchmark = False



    torch.backends.cudnn.deterministic = True



same_seeds(0)

## Read Data



- {train/test}set:



  - List of dicts with the following keys:



   - ID (string)



   - Sentence (string)



   - Aspect (string)



   - AspectFromTo (string): start point#end point



   - Category (string) : "食物#品質"



   - Opinion (string)



   - OpinionFromTo (string):  start point#end point





   - Intensity (string) :  "6.17#6.33"







- vaild set:



  - List of[ID, Sentence] 


In [28]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

# 读取数据函数
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    df = pd.json_normalize(data)
    return df


In [29]:
# 5 折交叉驗證
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 遍歷每一折
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
    print(f"Fold {fold + 1}:")
    
    # 獲取訓練集和驗證集
    train_fold_df = train_df.iloc[train_idx]
    val_fold_df = train_df.iloc[val_idx]
    
    # 顯示每折的數據大小
    print(f"Training Set Size: {len(train_fold_df)}")
    print(f"Validation Set Size: {len(val_fold_df)}")

    # 在這裡進行訓練和驗證的邏輯（如模型訓練）
    # 模型訓練的代碼應該寫在這裡
    
# 顯示測試集大小
print(f"Test Set Size: {len(test_df)}")


Fold 1:
Training Set Size: 4356
Validation Set Size: 1089
Fold 2:
Training Set Size: 4356
Validation Set Size: 1089
Fold 3:
Training Set Size: 4356
Validation Set Size: 1089
Fold 4:
Training Set Size: 4356
Validation Set Size: 1089
Fold 5:
Training Set Size: 4356
Validation Set Size: 1089
Test Set Size: 605


## Load Model and Tokenizer


### 創建自定義的 Dataset 類別

我們需要創建一個繼承自 torch.utils.data.Dataset 的類別，來處理資料並提供給 DataLoader。

In [30]:
from torch.utils.data import Dataset



class TripletExtractionDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels, intensity_labels):
        self.input_ids = inputs['input_ids']
        self.attention_mask = inputs['attention_mask']
        self.labels = labels
        self.intensity_labels = intensity_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'intensity_labels': torch.tensor(self.intensity_labels[idx], dtype=torch.float)
        }



### Tokenize Data



将数据准备为 BERT 输入格式

您可以提取 Sentence 字段并将其分词。为了准备 BERT 的输入，可以根据需要将 Aspect 和 Opinion 部分组合到句子中。这里假设要为每个 Sentence 生成分词结果并对齐其 Aspect 和 Opinion 信息：

In [ ]:
# 確認標籤數量
unique_labels = set()

for _, row in train_df.iterrows():
    for category in row['Category']:
        unique_labels.add(category)

# for i in unique_labels:

    # print(i)

num_labels = len(unique_labels)
#print("num_labels:", num_labels)


In [ ]:
# 定义标签映射

label_to_id = {'O': 0, 'B-Aspect': 1, 'I-Aspect': 2, 'B-Opinion': 3, 'I-Opinion': 4}



def encode_labels(text, aspects, opinions, aspect_positions, opinion_positions, intensities):
    # 使用相同的编码过程，确保 tokens 和 offset_mapping 对齐
    encoding = tokenizer(text, return_offsets_mapping=True, truncation=True, max_length=32, add_special_tokens=True)
    tokens = encoding.tokens()
    offset_mapping = encoding['offset_mapping']
    labels = ['O'] * len(tokens)
    # 初始化 intensity_labels，每个 token 对应一个 [Valence, Arousal]，初始为 [5.0, 5.0]
    intensity_labels = [[5.0, 5.0] for _ in range(len(tokens))]

    # 创建一个 mapping，将每个字符位置映射到对应的 token 索引
    char_to_token_map = {}
    for idx, (start, end) in enumerate(offset_mapping):
        if start == end:
            continue  # 跳过特殊标记或填充部分
        for char_pos in range(start, end):
            char_to_token_map[char_pos] = idx

    # 处理 Aspect 标签
    for aspect, pos in zip(aspects, aspect_positions):
        start_char, end_char = map(int, pos.split('#'))
        start_char -= 1  # 调整为 0-based 索引
        # 获取起始和结束 token 索引
        try:
            start_token_idx = char_to_token_map[start_char]
            end_token_idx = char_to_token_map[end_char - 1]
        except KeyError:
            continue  # 如果字符位置不在映射中，跳过
        labels[start_token_idx] = 'B-Aspect'
        for idx in range(start_token_idx + 1, end_token_idx + 1):
            labels[idx] = 'I-Aspect'

    # 处理 Opinion 标签，并赋予对应的情感强度
    for opinion, pos, intensity in zip(opinions, opinion_positions, intensities):
        start_char, end_char = map(int, pos.split('#'))
        start_char -= 1  # 调整为 0-based 索引
        valence, arousal = map(float, intensity.split('#'))  # 解析情感强度值
        try:
            start_token_idx = char_to_token_map[start_char]
            end_token_idx = char_to_token_map[end_char - 1]
        except KeyError:
            continue
        labels[start_token_idx] = 'B-Opinion'
        intensity_labels[start_token_idx] = [valence, arousal]
        for idx in range(start_token_idx + 1, end_token_idx + 1):
            labels[idx] = 'I-Opinion'
            intensity_labels[idx] = [valence, arousal]

    # 将标签转换为ID
    label_ids = [label_to_id[label] for label in labels]
    return label_ids, intensity_labels



def data_generator(df):
    # 处理整个数据集
    inputs = {'input_ids': [], 'attention_mask': []}
    label_ids_list = []
    intensity_labels_list = []

    for idx, row in df.iterrows():
        text = row['Sentence']
        aspects = row['Aspect']
        opinions = row['Opinion']
        aspect_positions = row['AspectFromTo']
        opinion_positions = row['OpinionFromTo']
        intensities = row['Intensity']  # 获取情感强度

        # 分词并编码
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=32, return_offsets_mapping=True)

        # 标签编码，获取 label_ids 和 intensity_labels
        label_ids, intensity_labels = encode_labels(text, aspects, opinions, aspect_positions, opinion_positions, intensities)

        # 填充或截断标签
        seq_length = len(encoding['input_ids'])
        if len(label_ids) < seq_length:
            label_ids += [label_to_id['O']] * (seq_length - len(label_ids))
            intensity_labels += [[0.0, 0.0]] * (seq_length - len(intensity_labels))
        else:
            label_ids = label_ids[:seq_length]
            intensity_labels = intensity_labels[:seq_length]

        # 移除 offset_mapping，因后续不需要
        encoding.pop('offset_mapping')

        # 添加到列表
        inputs['input_ids'].append(encoding['input_ids'])
        inputs['attention_mask'].append(encoding['attention_mask'])
        label_ids_list.append(label_ids)
        intensity_labels_list.append(intensity_labels)

    # 创建 Dataset
    dataset = TripletExtractionDataset(inputs, label_ids_list, intensity_labels_list)
    return dataset


# Train

### Define ASTE model

In [48]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel

class ASTEModel(BertPreTrainedModel):
    def __init__(self, config):
        super(ASTEModel, self).__init__(config)
        self.num_labels = config.num_labels  # 序列标注的标签数量
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # 序列标注层
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)
        # 情感强度回归层
        self.regressor = nn.Linear(config.hidden_size, 2)  # 输出 Valence 和 Arousal

        self.init_weights()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None, intensity_labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = self.dropout(outputs.last_hidden_state)  # [batch_size, seq_len, hidden_size]
    
        # 序列标注的 logits
        logits = self.classifier(sequence_output)  # [batch_size, seq_len, num_labels]
    
        total_loss = None
    
        # 计算序列标注的损失
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            active_loss = attention_mask.view(-1) == 1  # 只计算非填充部分的损失
            active_logits = logits.view(-1, self.num_labels)[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss = loss_fct(active_logits, active_labels)

            seq_loss  = loss
    
        # 计算情感强度的损失
        if intensity_labels is not None:
            # 获取观点词的位置
            opinion_mask = ((labels == 3) | (labels == 4)) & attention_mask  # [batch_size, seq_len]
            # 将张量重塑为 [batch_size * seq_len, ...]
            batch_size, seq_len, hidden_size = sequence_output.size()
            sequence_output_flat = sequence_output.view(-1, hidden_size)  # [batch_size * seq_len, hidden_size]
            opinion_mask_flat = opinion_mask.view(-1)  # [batch_size * seq_len]
            # 提取观点词的隐藏状态
            opinion_outputs = sequence_output_flat[opinion_mask_flat]  # [num_opinions, hidden_size]
            if opinion_outputs.size(0) > 0:
                # 预测情感强度
                intensity_preds = self.regressor(opinion_outputs)  # [num_opinions, 2]
                # 取对应的情感强度标签
                intensity_labels_flat = intensity_labels.view(-1, 2)  # [batch_size * seq_len, 2]
                intensity_targets = intensity_labels_flat[opinion_mask_flat]  # [num_opinions, 2]
                # 计算回归损失
                loss_mse = nn.MSELoss()
                regression_loss = loss_mse(intensity_preds, intensity_targets)
                total_loss = seq_loss * 1.0 + regression_loss * 0.1 if total_loss is not None else regression_loss
    
        output = (logits,)
        return ((total_loss,) + output) if total_loss is not None else output
    


### 辅助函数的定义
1. 提取 BIO 标签对应的实体（方面和观点）:extract_spans
2. 提取预测的三元组
3. 提取真实的三元组
4. 计算评估指标

In [ ]:
def extract_spans(tags, tokens, label_prefix):
    spans = []
    i = 0
    while i < len(tags):
        if tags[i] == f'B-{label_prefix}':
            start = i
            i += 1
            while i < len(tags) and tags[i] == f'I-{label_prefix}':
                i += 1
            end = i  # 结束位置的下一个索引
            span_tokens = tokens[start:end]
            spans.append((start, end, span_tokens))
        else:
            i += 1
    return spans

def extract_triplets(tokens, tags, model, input_ids, attention_mask):
    aspect_spans = extract_spans(tags, tokens, 'Aspect')
    opinion_spans = extract_spans(tags, tokens, 'Opinion')

    binary_aspect_opinion = []  # 1. Aspect-Opinion 二元组
    v_triplets = []            # 2. 包含 Valence 的 triplets
    a_triplets = []            # 3. 包含 Arousal 的 triplets
    full_triplets = []         # 4. 原始的 Aspect-Opinion-Valence-Arousal 三元组

    # 获取模型的序列输出
    with torch.no_grad():
        outputs = model.bert(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        sequence_output = outputs.last_hidden_state.squeeze(0)  # [seq_len, hidden_size]

    for opinion_span in opinion_spans:
        start, end, opinion_tokens = opinion_span

        # 获取观点词的隐藏状态
        opinion_hidden_states = sequence_output[start:end]

        # 计算平均表示
        opinion_representation = opinion_hidden_states.mean(dim=0)  # [hidden_size]

        # 预测情感强度
        intensity_pred = model.regressor(opinion_representation.unsqueeze(0))  # [1, 2]
        intensity_pred = intensity_pred.squeeze(0).detach().cpu().numpy()

        # 原本只有round
        valence = int(round(intensity_pred[0] * 10))
        arousal = int(round(abs(intensity_pred[1] * 10)))

        opinion_text = ''.join(opinion_tokens).replace('##', '')

        if len(aspect_spans) == 0:
            aspect_text = ''
            binary_aspect_opinion.append((aspect_text, opinion_text))
            v_triplets.append((aspect_text, opinion_text, valence))
            a_triplets.append((aspect_text, opinion_text, arousal))
            full_triplets.append((aspect_text, opinion_text, valence, arousal))
        else:
            for aspect_span in aspect_spans:
                a_start, a_end, aspect_tokens = aspect_span
                aspect_text = ''.join(aspect_tokens).replace('##', '')
                binary_aspect_opinion.append((aspect_text, opinion_text))
                v_triplets.append((aspect_text, opinion_text, valence))
                a_triplets.append((aspect_text, opinion_text, arousal))
                full_triplets.append((aspect_text, opinion_text, valence, arousal))

    return binary_aspect_opinion, v_triplets, a_triplets, full_triplets


def extract_gold_triplets(tokens, tags, intensity_labels):
    aspect_spans = extract_spans(tags, tokens, 'Aspect')
    opinion_spans = extract_spans(tags, tokens, 'Opinion')

    binary_aspect_opinion = []  # 1. Aspect-Opinion 二元组
    v_triplets = []            # 2. 包含 Valence 的 triplets
    a_triplets = []            # 3. 包含 Arousal 的 triplets
    full_triplets = []         # 4. 原始的 Aspect-Opinion-Valence-Arousal 三元组

    for opinion_span in opinion_spans:
        start, end, opinion_tokens = opinion_span

        # 获取观点词的情感强度标签
        intensity_values = intensity_labels[start:end]  # [span_len, 2]
        intensity_values = intensity_values.mean(dim=0).detach().cpu().numpy()

        valence = int(round(intensity_values[0]))
        arousal = int(round(intensity_values[1]))

        opinion_text = ''.join(opinion_tokens).replace('##', '')

        if len(aspect_spans) == 0:
            aspect_text = ''
            binary_aspect_opinion.append((aspect_text, opinion_text))
            v_triplets.append((aspect_text, opinion_text, valence))
            a_triplets.append((aspect_text, opinion_text, arousal))
            full_triplets.append((aspect_text, opinion_text, valence, arousal))
        else:
            for aspect_span in aspect_spans:
                a_start, a_end, aspect_tokens = aspect_span
                aspect_text = ''.join(aspect_tokens).replace('##', '')
                binary_aspect_opinion.append((aspect_text, opinion_text))
                v_triplets.append((aspect_text, opinion_text, valence))
                a_triplets.append((aspect_text, opinion_text, arousal))
                full_triplets.append((aspect_text, opinion_text, valence, arousal))

    return binary_aspect_opinion, v_triplets, a_triplets, full_triplets


def compute_metrics(pred_triplets, gold_triplets):

    
    pred_set = set(pred_triplets)
    gold_set = set(gold_triplets)

    true_positives = pred_set & gold_set
    # if(true_positives):
    #     print(pred_triplets[:3])
    #     print(gold_triplets[:3])
    # print(true_positives)
        
    precision = len(true_positives) / len(pred_set) if len(pred_set) > 0 else 0.0
    recall = len(true_positives) / len(gold_set) if len(gold_set) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1


### Training

In [ ]:
from transformers import BertConfig
from torch.amp import autocast, GradScaler
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from sklearn.model_selection import KFold
import os
import numpy as np
from tqdm import tqdm
import torch
import pandas as pd
import json
from torch.utils.tensorboard import SummaryWriter
from transformers import get_scheduler


# 初始化
scaler = GradScaler()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 超参数设置
best_val_loss = float('inf')
best_avg_loss = float('inf')
patience = 15
patience_counter = 0
min_delta = 0.001
max_epochs = 1000  # 设置一个最大 epoch 数，防止无限循环

label_to_id = {'O': 0, 'B-Aspect': 1, 'I-Aspect': 2, 'B-Opinion': 3, 'I-Opinion': 4}
id_to_label = {v: k for k, v in label_to_id.items()}
num_labels = len(label_to_id)

# 加载数据
data_path = r'E:\NYCU-Project\Class\NLP\Dimensional ASTE\NYCU_NLP_113A_Dataset\NYCU_NLP_113A_TrainingSet.json'
df = read_data(data_path)

# 分割测试集
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

test_dataset = data_generator(test_df)
test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=32)

print(f"Test Set Size: {len(test_df)}")

# 使用BERT快速標記器來自動對齊標記和標籤

tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese') # 使用支持 offset_mapping 的 BertTokenizerFast
# K折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_df)):
    print(f"Fold {fold + 1}:")
    writer = SummaryWriter(log_dir=f"./tensorboard_logs/fold_{fold + 1}")


    train_fold_df = train_val_df.iloc[train_idx]
    val_fold_df = train_val_df.iloc[val_idx]

    # 显示每折的数据大小
    print(f"Training Set Size: {len(train_fold_df)}")
    print(f"Validation Set Size: {len(val_fold_df)}")

    Train_dataset = data_generator(train_fold_df)
    val_dataset = data_generator(val_fold_df)
    
    # 创建 DataLoader
    train_loader = DataLoader(Train_dataset, sampler=RandomSampler(Train_dataset), batch_size=32)
    val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=32)

    # 加载模型和配置
    config = BertConfig.from_pretrained('bert-base-chinese', num_labels=num_labels)
    model = ASTEModel.from_pretrained('bert-base-chinese', config=config)
    model.to(device)

    # 定义分层学习率
    optimizer_grouped_parameters = [
        {'params': model.bert.encoder.layer[:6].parameters(), 'lr': 1e-5},  # BERT 的底层 Transformer 层
        {'params': model.bert.encoder.layer[6:].parameters(), 'lr': 2e-5},  # BERT 的高层 Transformer 层
        {'params': model.classifier.parameters(), 'lr': 1e-4},  # 序列标注层（任务相关层）
        {'params': model.regressor.parameters(), 'lr': 1e-4},  # 回归层（任务相关层）
    ]
    optimizer = AdamW(optimizer_grouped_parameters, weight_decay=0.01)

    num_training_steps = max_epochs * len(train_loader)
    num_warmup_steps = 1000    # 预热步数为总步数的10%

    scheduler = get_scheduler(
        "linear",  
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
    output_dir = f'./saved_model_fold_{fold + 1}/'

    epochs = 0  # 重置 epoch 计数器
    patience_counter = 0  # 重置耐心计数器

    # 将训练循环修改为 while 循环
    while True:
        epochs += 1  # 手动计数 epochs
        if epochs > max_epochs:
            print("达到最大 epoch 数，停止训练。")
            break

        model.train()
        total_loss = 0

        for step, batch in enumerate(tqdm(train_loader)):
            batch_input_ids = batch['input_ids'].to(device)
            batch_attention_masks = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)
            batch_intensity_labels = batch['intensity_labels'].to(device)

            optimizer.zero_grad()

            with autocast("cuda"):
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_masks,
                    labels=batch_labels,
                    intensity_labels=batch_intensity_labels
                )
            loss = outputs[0]

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        # 在每个 epoch 结束后更新学习率
        scheduler.step()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epochs}, Average Loss: {avg_loss}, Learning Rate: {scheduler.get_last_lr()[0]}")

        # 验证步骤
        model.eval()
        total_eval_loss = 0

        all_pred_binary = []
        all_gold_binary = []
        all_pred_v_triplets = []
        all_gold_v_triplets = []
        all_pred_a_triplets = []
        all_gold_a_triplets = []
        all_pred_triplets = []
        all_gold_triplets = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader):
                batch_input_ids = batch['input_ids'].to(device)
                batch_attention_masks = batch['attention_mask'].to(device)
                batch_labels = batch['labels'].to(device)
                batch_intensity_labels = batch['intensity_labels'].to(device)

                # with autocast("cuda"):
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_masks,
                    labels=batch_labels,
                    intensity_labels=batch_intensity_labels
                )
                loss = outputs[0]
                total_eval_loss += loss.item()

                logits = outputs[1]
                predictions = torch.argmax(logits, dim=2)

                for i in range(batch_input_ids.size(0)):
                    input_ids = batch_input_ids[i]
                    pred_labels = predictions[i]
                    gold_labels = batch_labels[i]
                    attention_mask = batch_attention_masks[i]
                    intensity_labels = batch_intensity_labels[i]

                    tokens = tokenizer.convert_ids_to_tokens(input_ids.cpu().numpy())
                    pred_tags = [id_to_label[label_id.item()] for label_id in pred_labels]
                    gold_tags = [id_to_label[label_id.item()] for label_id in gold_labels]

                    pred_binary, pred_v_triplets, pred_a_triplets, pred_triplets = extract_triplets(tokens, pred_tags, model, input_ids, attention_mask)
                    gold_binary, gold_v_triplets, gold_a_triplets, gold_triplets = extract_gold_triplets(tokens, gold_tags, intensity_labels)
                    # 可以打印或记录预测结果进行分析
                    # if(i<5):
                    #     print("pred_triplets:", pred_triplets)
                    #     print("gold_triplets:", gold_triplets)
                    
                    all_pred_binary.extend(pred_binary)
                    all_gold_binary.extend(gold_binary)
                    
                    all_pred_v_triplets.extend(pred_v_triplets)
                    all_gold_v_triplets.extend(gold_v_triplets)
                    
                    all_pred_a_triplets.extend(pred_a_triplets)
                    all_gold_a_triplets.extend(gold_a_triplets)                    

                    all_pred_triplets.extend(pred_triplets)
                    all_gold_triplets.extend(gold_triplets)

        _, _, binary_f1 = compute_metrics(all_pred_binary, all_gold_binary)
        _, _, v_f1 = compute_metrics(all_pred_v_triplets, all_gold_v_triplets)
        _, _, a_f1 = compute_metrics(all_pred_a_triplets, all_gold_a_triplets)
        precision, recall, f1 = compute_metrics(all_pred_triplets, all_gold_triplets)
        # 将 all_pred_triplets 的前三项追加到 txt 文件
        output_file_path = "./log/pred_triplets_output.txt"  # 替换为你的文件路径

        try:
            with open(output_file_path, 'a', encoding='utf-8') as f:
                for i in range(3):
                    f.write(f"{all_pred_triplets[i]},{all_gold_triplets[i]}\n")
                    f.write(f"---------------------------------------------\n")

        except Exception as e:
            print(f"Failed to write to file: {e}")
            
        avg_val_loss = total_eval_loss / len(val_loader)
        print(f"Val Loss: {avg_val_loss}, binary_f1: {binary_f1:.4f}, v_f1: {v_f1:.4f}, a_f1: {a_f1:.4f}, F1-score: {f1:.4f}")

        # 记录验证 Loss 和 F1 Score 到 TensorBoard
        writer.add_scalar("Loss/Validation", avg_val_loss, global_step=epochs)
        writer.add_scalar("Metrics/F1", f1, global_step=epochs)
        writer.add_scalar("Metrics/Precision", precision, global_step=epochs)
        writer.add_scalar("Metrics/Recall", recall, global_step=epochs)
        
        # 检查验证损失是否改善
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            patience_counter = 0

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            model.save_pretrained(output_dir)
            tokenizer.save_pretrained(output_dir)
        else:
            patience_counter += 1
            print(f"验证损失未改善，耐心值计数：{patience_counter}")
            if patience_counter >= patience:
                print("验证损失在连续多个 epoch 中未改善，停止训练。")
                break

    writer.close()
    print(f"Fold {fold + 1} 训练完成。")
    # 重置计数器
    epochs = 0
    patience_counter = 0

print("全部训练完成")


Test Set Size: 605
Fold 1:
Training Set Size: 4356
Validation Set Size: 1089


Some weights of ASTEModel were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight', 'regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 137/137 [00:20<00:00,  6.77it/s]


Epoch 1, Average Loss: 0.2327087053512461, Learning Rate: 1e-08


100%|██████████| 35/35 [00:18<00:00,  1.94it/s]


[('[CLS]', '港'), ('的', '港'), ('表', '港')]
[('港點的表現', '蠻不錯'), ('湯汁', '微酸的'), ('魚肉', '微酸的')]
{('肉', '嫩'), ('肉', '脆')}
[('[CLS]', '港', -3), ('的', '港', -3), ('表', '港', -3)]
[('港點的表現', '蠻不錯', 6), ('湯汁', '微酸的', 6), ('魚肉', '微酸的', 6)]
{('肉', '嫩', 6), ('肉', '脆', 6)}
Top 3 predicted triplets appended to ./log/pred_triplets_output.txt.
Val Loss: 0.35875006530966075, binary_f1: 0.0002, v_f1: 0.0001, a_f1: 0.0000, F1-score: 0.0000


...

[('', '[CLS]'), ('', '肉'), ('', '又軟')]
[('肉質', '軟嫩'), ('酸辣湯', '超級普通'), ('紅燒肉', '真的優秀')]
{('', '鮮味十足'), ('麵', '還可以')}
Top 3 predicted triplets appended to ./log/pred_triplets_output.txt.
Val Loss: 0.02230956751321043, binary_f1: 0.0004, v_f1: 0.0000, a_f1: 0.0000, F1-score: 0.0000
验证损失未改善，耐心值计数：14


100%|██████████| 137/137 [00:24<00:00,  5.66it/s]


Epoch 15, Average Loss: 0.2501831861097284, Learning Rate: 1.5000000000000002e-07


100%|██████████| 35/35 [00:22<00:00,  1.54it/s]

[('', '[CLS]'), ('', '肉'), ('', '又軟')]
[('肉質', '軟嫩'), ('酸辣湯', '超級普通'), ('紅燒肉', '真的優秀')]
{('', '鮮味十足'), ('麵', '還可以')}
Top 3 predicted triplets appended to ./log/pred_triplets_output.txt.
Val Loss: 0.021695455070585012, binary_f1: 0.0004, v_f1: 0.0000, a_f1: 0.0000, F1-score: 0.0000
验证损失未改善，耐心值计数：15
验证损失在连续多个 epoch 中未改善，停止训练。
Fold 5 训练完成。
全部训练完成


# Testing

 1. 使用自定义的 ASTEModel 进行预测
 2. 提取实体并格式化输出

In [55]:
import torch
from transformers import BertConfig, BertTokenizerFast

# 定义标签映射
label_to_id = {'O': 0, 'B-Aspect': 1, 'I-Aspect': 2, 'B-Opinion': 3, 'I-Opinion': 4}
id_to_label = {v: k for k, v in label_to_id.items()}

# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载配置和自定义模型 ASTEModel
config = BertConfig.from_pretrained('./saved_model/', num_labels=len(label_to_id))
model = ASTEModel.from_pretrained('./saved_model/', config=config)
model.to(device)
model.eval()

# 加载 tokenizer
tokenizer = BertTokenizerFast.from_pretrained('./saved_model/')

def predict(sentence):
    encoding = tokenizer(
        sentence,
        return_offsets_mapping=True,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt',
        add_special_tokens=True
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    offset_mapping = encoding['offset_mapping'][0]  # batch_size=1

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]  # 获取预测的标签 logits
    predictions = torch.argmax(logits, dim=2)  # [1, seq_len]
    predictions = predictions[0].cpu().numpy()  # 转换为 numpy 数组

    # 将 label id 转换为 label 名称
    predicted_labels = [id_to_label[int(label_id)] for label_id in predictions]

    # 将 tokens 转换回文字
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # 去除填充部分
    effective_length = (attention_mask[0] == 1).sum().item()
    tokens = tokens[:effective_length]
    predicted_labels = predicted_labels[:effective_length]
    offset_mapping = offset_mapping[:effective_length]

    # 获取模型的序列输出，用于情感强度预测
    with torch.no_grad():
        bert_outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state[0, :effective_length, :]  # [seq_len, hidden_size]

    return tokens, predicted_labels, offset_mapping, sequence_output

def get_entities(tokens, labels, offsets, sequence_output):
    aspects = []
    opinions = []
    idx = 0
    while idx < len(labels):
        label = str(labels[idx])  # 确保 label 是字符串
        if label == 'B-Aspect':
            aspect_tokens = [tokens[idx]]
            aspect_start = offsets[idx][0].item()
            aspect_end = offsets[idx][1].item()
            aspect_positions = [idx]
            idx += 1
            while idx < len(labels) and str(labels[idx]) == 'I-Aspect':
                aspect_tokens.append(tokens[idx])
                aspect_end = offsets[idx][1].item()
                aspect_positions.append(idx)
                idx += 1
            aspect_text = ''.join([token.lstrip('##') for token in aspect_tokens])
            aspects.append({'text': aspect_text, 'start': aspect_start, 'end': aspect_end, 'positions': aspect_positions})
        elif label == 'B-Opinion':
            opinion_tokens = [tokens[idx]]
            opinion_start = offsets[idx][0].item()
            opinion_end = offsets[idx][1].item()
            opinion_positions = [idx]
            idx += 1
            while idx < len(labels) and str(labels[idx]) == 'I-Opinion':
                opinion_tokens.append(tokens[idx])
                opinion_end = offsets[idx][1].item()
                opinion_positions.append(idx)
                idx += 1
            opinion_text = ''.join([token.lstrip('##') for token in opinion_tokens])

            # 获取观点词的隐藏状态
            opinion_hidden_states = sequence_output[opinion_positions, :]  # [opinion_len, hidden_size]
            # 计算平均表示
            opinion_representation = opinion_hidden_states.mean(dim=0)  # [hidden_size]
            # 预测情感强度
            with torch.no_grad():
                intensity_pred = model.regressor(opinion_representation.unsqueeze(0))  # [1, 2]
                intensity_pred = intensity_pred.squeeze(0).detach().cpu().numpy()
                # 四舍五入为整数
                valence = int(round(intensity_pred[0]))
                arousal = int(round(intensity_pred[1]))
                intensity = f"{valence}#{arousal}"

            opinions.append({
                'text': opinion_text,
                'start': opinion_start,
                'end': opinion_end,
                'intensity': intensity
            })
        else:
            idx += 1
    return aspects, opinions

def format_output(sentence_id, aspects, opinions):
    triplets = []
    for aspect in aspects:
        for opinion in opinions:
            # 使用预测的情感强度
            intensity = opinion['intensity']
            triplet = f"({aspect['text']}, {opinion['text']}, {intensity})"
            triplets.append(triplet)
    triplets_str = ' '.join(triplets)
    return f"{sentence_id}\t{triplets_str}"

# 示例句子
sentence_id = 'R3530:S002'
sentence = '今天點了牛雜蘿蔔煲微辣超夠味很好吃，份量蠻多的。'

# 预测
tokens, predict_labels, offsets, sequence_output = predict(sentence)

# 提取实体并预测情感强度
aspects, opinions = get_entities(tokens, predict_labels, offsets, sequence_output)
print("Tokens:", tokens)
print("Predicted Labels:", predict_labels)
print("Aspects:", aspects)
print("Opinions:", opinions)

# 格式化输出
output_line = format_output(sentence_id, aspects, opinions)
print('ID\tTriplets')
print(output_line)


Tokens: ['[CLS]', '今', '天', '點', '了', '牛', '雜', '蘿', '蔔', '煲', '微', '辣', '超', '夠', '味', '很', '好', '吃', '，', '份', '量', '蠻', '多', '的', '。', '[SEP]']
Predicted Labels: ['O', 'O', 'O', 'O', 'O', 'B-Aspect', 'I-Aspect', 'I-Aspect', 'I-Aspect', 'I-Aspect', 'B-Opinion', 'I-Opinion', 'B-Opinion', 'I-Opinion', 'I-Opinion', 'B-Opinion', 'I-Opinion', 'I-Opinion', 'O', 'B-Aspect', 'I-Aspect', 'B-Opinion', 'I-Opinion', 'I-Opinion', 'O', 'O']
Aspects: [{'text': '牛雜蘿蔔煲', 'start': 4, 'end': 9, 'positions': [5, 6, 7, 8, 9]}, {'text': '份量', 'start': 18, 'end': 20, 'positions': [19, 20]}]
Opinions: [{'text': '微辣', 'start': 9, 'end': 11, 'intensity': '1#1'}, {'text': '超夠味', 'start': 11, 'end': 14, 'intensity': '1#1'}, {'text': '很好吃', 'start': 14, 'end': 17, 'intensity': '1#1'}, {'text': '蠻多的', 'start': 20, 'end': 23, 'intensity': '1#1'}]
ID	Triplets
R3530:S002	(牛雜蘿蔔煲, 微辣, 1#1) (牛雜蘿蔔煲, 超夠味, 1#1) (牛雜蘿蔔煲, 很好吃, 1#1) (牛雜蘿蔔煲, 蠻多的, 1#1) (份量, 微辣, 1#1) (份量, 超夠味, 1#1) (份量, 很好吃, 1#1) (份量, 蠻多的, 1#1)
